<a href="https://colab.research.google.com/github/AlexRaudvee/MultiArchPDD-CV/blob/main/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup of environment

In [1]:
import os
import shutil
import zipfile
from pathlib import Path
from google.colab import drive

def mount_google_drive(mount_point: Path = Path('/content/drive')) -> Path:
    """Mounts Google Drive and returns the mount point."""
    drive.mount(str(mount_point))
    return mount_point

def extract_zip(zip_path: Path, extract_to: Path) -> None:
    """Extracts a zip file to the given directory."""
    if not zip_path.is_file():
        raise FileNotFoundError(f"Could not find zip file at {zip_path}")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(str(extract_to))

def move_contents(src_dir: Path, dst_dir: Path) -> None:
    """
    Moves everything from src_dir into dst_dir.
    Overwrites any existing files or folders of the same name.
    Cleans up the now-empty src_dir at the end.
    """
    if not src_dir.is_dir():
        raise FileNotFoundError(f"{src_dir} does not exist")
    for item in src_dir.iterdir():
        target = dst_dir / item.name
        if target.exists():
            print(f"Warning: {target} already exists, overwriting")
            if target.is_dir():
                shutil.rmtree(target)
            else:
                target.unlink()
        shutil.move(str(item), str(target))
    src_dir.rmdir()

def setup_directories(*dirs: Path) -> None:
    """Ensures that each directory in `dirs` exists."""
    for d in dirs:
        d.mkdir(parents=True, exist_ok=True)

def zip_folder(folder_path: Path, output_path: Path) -> None:
    """
    Recursively zip the contents of folder_path into a .zip file at output_path.
    """
    with zipfile.ZipFile(output_path, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for fname in files:
                fpath = Path(root) / fname
                arcname = fpath.relative_to(folder_path)
                zipf.write(str(fpath), arcname)

In [2]:
# ——— Constants ———
DRIVE_MOUNT_POINT = Path('/content/drive')
ZIP_PATH            = DRIVE_MOUNT_POINT / 'MyDrive/.colab.zip'
EXTRACT_TO          = Path('/content')
SRC_DIR             = EXTRACT_TO / '.colab'
DST_DIR             = EXTRACT_TO
DISTILLED_DIR       = EXTRACT_TO / 'data' / 'Distilled'
MODEL_DIR           = EXTRACT_TO / 'data' / 'checkpoints'
ASSETS_DIR          = EXTRACT_TO / 'assets' / 'viz_synthetic'

# ——— SetUp ———
mount_google_drive(DRIVE_MOUNT_POINT)
extract_zip(ZIP_PATH, EXTRACT_TO)
move_contents(SRC_DIR, DST_DIR)
setup_directories(DISTILLED_DIR)
setup_directories(ASSETS_DIR)
setup_directories(MODEL_DIR)

Mounted at /content/drive


In [3]:
!pip install matplotlib

### Launch of Dataset Distillation

In [ ]:
!python main.py meta-model-matching \
    --dataset mnist \
    --model convnet \
    --batch-size 32 \
    --ipc 1 \
    --P 5 \
    --K 50 \
    --T 1 \
    --lr-model 1e-3 \
    --lr-syn-data 1e-2 \
    --syn-optimizer momentum \
    --inner-optimizer momentum \
    --debug True \
    --out-dir data/Distilled \
    --ckpt-dir data/checkpoints

[Dataloader]:
     - Loading...
     - Done.
[Distillator]:
/usr/local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Stage 1/5:   0%|                                         | 0/50 [00:00<?, ?it/s]T Loss=2.3053462505340576
g_norm = tensor(0.1037, grad_fn=<LinalgVectorNormBackward0>)
alpha_t= 0.0010000233305618167
K Loss      =2303.702880859375
||∇_X meta|| = 4.426256055012345e-05
ΔX norm: 4.4262552023610624e-07
Saved synthetic image grid to assets/debug/synthetic.png
Stage 1/5:   2%|▋                                | 1/50 [00:00<00:24,  2.02it/s]T Loss=2.3036932945251465
g_norm = tensor(0.0792, grad_fn=<LinalgVectorNormBackward0>)
alpha_t= 0.0010000233305618167
K Loss      =2304.285400390625
||∇_X meta|| = 3.970310717704706e-05
ΔX norm: 3.9703087395537295e-07
Stage 1/5:   4%|█▎                               | 2/50 [00:00<00

In [ ]:
!python main.py meta-model-matching \
    --dataset cifar10 \
    --model convnet \
    --batch-size 32 \
    --ipc 10 \
    --P 10 \
    --K 300 \
    --T 5 \
    --lr-model 1e-3 \
    --lr-syn-data 1e-2 \
    --regularisation 1 \
    --syn-optimizer momentum \
    --inner-optimizer momentum \
    --debug True \
    --out-dir data/Distilled \
    --ckpt-dir data/checkpoints

[Dataloader]:
     - Loading...
     - Done.
[Distillator]:
/usr/local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Stage 1/10:   0%|                                       | 0/300 [00:00<?, ?it/s]T Loss=2.305534601211548
g_norm = tensor(0.1072, grad_fn=<LinalgVectorNormBackward0>)
alpha_t= 0.0010000233305618167
T Loss=2.3032116889953613
g_norm = tensor(0.0963, grad_fn=<LinalgVectorNormBackward0>)
alpha_t= 0.0010000233305618167
T Loss=2.3034181594848633
g_norm = tensor(0.0868, grad_fn=<LinalgVectorNormBackward0>)
alpha_t= 0.0010000233305618167
T Loss=2.3034520149230957
g_norm = tensor(0.0954, grad_fn=<LinalgVectorNormBackward0>)
alpha_t= 0.0010000233305618167
T Loss=2.303032636642456
g_norm = tensor(0.0990, grad_fn=<LinalgVectorNormBackward0>)
alpha_t= 0.0010000233305618167
K Loss      =229.1239471435547
||∇_X meta|| = 0.00

### Gradient Aggregation

In [4]:
!python main.py gradient-aggregation \
    --dataset mnist \
    --model convnet resnet10\
    --batch-size 32 \
    --ipc 1 \
    --P 5 \
    --K 50 \
    --T 1 \
    --lr-model 1e-3 \
    --lr-syn-data 1e-2 \
    --syn-optimizer momentum \
    --inner-optimizer momentum \
    --debug True \
    --out-dir data/Distilled \
    --ckpt-dir data/checkpoints

[Dataloader]:
     - Loading...
100% 9.91M/9.91M [00:01<00:00, 5.07MB/s]
100% 28.9k/28.9k [00:00<00:00, 133kB/s]
100% 1.65M/1.65M [00:06<00:00, 245kB/s]
100% 4.54k/4.54k [00:00<00:00, 13.2MB/s]
     - Done.
[Distillator]:
Stage 1/5:   0% 0/50 [00:00<?, ?it/s]     - Model 1: T Loss      =[tensor(2.3040, device='cuda:0', grad_fn=<DivBackward0>), tensor(2.6250, device='cuda:0', grad_fn=<DivBackward0>)]
     - Model 1: g_norm      = 0.08606026321649551
     - Model 1: alpha_t     = 0.0010000233305618167
     - Model 2: T Loss      =[tensor(2.3040, device='cuda:0', grad_fn=<DivBackward0>), tensor(2.6250, device='cuda:0', grad_fn=<DivBackward0>)]
     - Model 2: g_norm      = 506.1029357910156
     - Model 2: alpha_t     = 0.0010000233305618167
K Losses    =[tensor(2.2790, device='cuda:0', grad_fn=<AddBackward0>), tensor(2.2778, device='cuda:0', grad_fn=<AddBackward0>)]
||∇_X meta|| = 14.975476264953613
ΔX norm:       0.14975474774837494
Saved synthetic image grid to assets/debug/synthetic.p

In [ ]:
!python main.py gradient-aggregation \
    --dataset cifar10 \
    --model convnet resnet10\
    --batch-size 32 \
    --ipc 10 \
    --P 10 \
    --K 300 \
    --T 5 \
    --lr-model 1e-3 \
    --lr-syn-data 1e-2 \
    --regularisation 1 \
    --syn-optimizer momentum \
    --inner-optimizer momentum \
    --debug True \
    --out-dir data/Distilled \
    --ckpt-dir data/checkpoints

### Benchmarking of Distilled Dataset (dev - accuracy performance)

In [ ]:
!python main.py benchmark \
    --distilled-path data/Distilled/meta-model-matching_cifar10_convnet.pt \
    --benchmark-mode synthetic \
    --model convnet \
    --syn-batch-size 64 \
    --test-batch-size 64 \
    --lr 1e-3  \
    --epochs-per-stage 10 \
    --till-stage 10 \
    --real-size 1000

[Benchmarker]:
     - Using device: cpu
     - Loading distilled data from data/Distilled/meta-model-matching_cifar10_convnet.pt
     - Total synthetic examples = 1000; real subset size = 1000

     - [Syn] Stage 1/10: 100 examples
       - Epoch 1/10 → loss 2.3099
       - Epoch 2/10 → loss 2.3013
       - Epoch 3/10 → loss 2.2967
       - Epoch 4/10 → loss 2.2881
       - Epoch 5/10 → loss 2.2760
       - Epoch 6/10 → loss 2.2452
       - Epoch 7/10 → loss 2.2044
       - Epoch 8/10 → loss 2.1363
       - Epoch 9/10 → loss 2.0434
       - Epoch 10/10 → loss 1.9242

     - [Syn] Stage 2/10: 100 examples
       - Epoch 1/10 → loss 1.8214
       - Epoch 2/10 → loss 1.6980
       - Epoch 3/10 → loss 1.5841
       - Epoch 4/10 → loss 1.4594
       - Epoch 5/10 → loss 1.3280
       - Epoch 6/10 → loss 1.2482
       - Epoch 7/10 → loss 1.0842
       - Epoch 8/10 → loss 0.9600
       - Epoch 9/10 → loss 0.9304
       - Epoch 10/10 → loss 0.8757

     - [Syn] Stage 3/10: 100 examples
       -